In [ ]:
%load_ext autoreload
%autoreload 2

# 4. Sentiment Analysis - Multilayer Perceptron

The Multilayer Perceptron (**MLP**) is considered one of the most basic building blocks for netural networks. While the simple Perceptron takes data vector as input and computes a single output value, MLP groups many perceptrons, so the output of the single layer is a new vector instead of a single output value.

In PyTorch, this is done simply by setting the number of output features in the <code>Linear</code> layer. Additionally, in MLP multiple layers are combined with a nonlinearity between each layer.

<img src="files/mlp.png" width="600" height="300" align="center"/>

When it comes to the **Sentiment Analysis task** we are solving, everything except the model itself, stays the same as in previous example: building the dataset, vectorizer, vocabulary, data loader, training loop and the evaluation. We are going to use the one-hot encoding to represent the text of the tweet, as in previous example.

## Setup

Firstly, set up the path to the (preprocessed) dataset

In [ ]:
# Path to the preprocessed data
import os

fileDir = os.path.dirname(os.path.realpath('__file__'))
absFilePathToPreprocessedDataset = os.path.join(fileDir, '../Data/training.1600000.processed.noemoticon_preprocessed.csv')
pathToPreprocessedDataset = os.path.abspath(os.path.realpath(absFilePathToPreprocessedDataset))
print (pathToPreprocessedDataset)

Choose the device to run the training on:

In [ ]:
device = "cpu"

Set the learning rate parameter:

In [ ]:
learningRate = 0.001

Set the size of the hidden layer for the MLP model:

In [ ]:
hidden_dim = 100

## Initialization

In [ ]:
import torch.nn as nn
import torch.optim as optim
from Common.TwitterDataset import TwitterDataset
from Models.ModelMLP import SentimentClassifierMLP

# Step #1: Instantiate the dataset
# instantiate the dataset
dataset = TwitterDataset.load_dataset_and_make_vectorizer(pathToPreprocessedDataset)
# get the vectorizer
vectorizer = dataset.get_vectorizer()

# Step #2: Instantiate the model
# instantiate the model
model = SentimentClassifierMLP(input_dim=len(vectorizer.text_vocabulary), hidden_dim=hidden_dim, output_dim=len(vectorizer.target_vocabulary))
# send model to appropriate device
model = model.to(device)

# Step #3: Instantiate the loss function
loss_func = nn.CrossEntropyLoss()

# Step #4: Instantiate the optimizer
optimizer = optim.Adam(model.parameters(), lr=learningRate)

## Training Loop

In [ ]:
from Common.Trainer import Trainer

sentiment_analysis_trainer = Trainer(
    dataset=dataset,
    model=model,
    loss_func=loss_func,
    optimizer=optimizer
)

In [ ]:
# setup the chosen number of epochs
num_epochs = 200
# setup the chosen batch size
batch_size = 16

report = sentiment_analysis_trainer.train(num_epochs=num_epochs, batch_size=batch_size, device=device)

## Evaluate the results

In [ ]:
from RunHelper import evaluate_model

# set the model in eval state
model.eval()

evaluate_model(sentiment_analysis_trainer, device, batch_size)

## Inference and classifying new data points

Let's do inference on the new data. This is another evaluation method to make qualitative judgement about whether the model is working.

Let's try the model on some examples:

In [ ]:
from RunHelper import run_examples

examples = [
    "This is a good day.",
    "I was very sad yesterday.",
    "This is a book."
]

run_examples(examples, model, vectorizer)

### More detailed evaluation on the Test Set

In [ ]:
from RunHelper import model_run_and_evaluate

model_run_and_evaluate(dataset, vectorizer, model)